In [179]:
import os
import tensorflow as tf
from tensorflow.image import resize
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from keras.metrics import  Recall, CategoricalAccuracy
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import entropy

from helpers.help import *
np.random.seed(0)

In [180]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'gaussian_ds')
label_dict={'Mild':1,'Moderate':1,'Proliferate_DR':1,'Severe':1,'No_DR':0}

# remove macOS file
folders = os.listdir(path)
# folders.remove('.DS_Store')

# get all the samples
array = []
for i in folders:
    detailPath = os.path.join(path,i)
    for j in os.listdir(detailPath):
        array.append([os.path.join(detailPath,j),label_dict[i.split('.')[0]]])

# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

3662

In [181]:
X,y=dataset[::,0],dataset[::,1]
y = y.astype(int)

#One hot encode the labels
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 10% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.1*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 11% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.11, random_state=42)

In [182]:
#We use the helper function to convert the data into tensorflow dataset objects. Note that , the repeat flag needs
#to be set only for the train set , which by default is true.
#The buid_dataset is a custom function that returns tensor batches
# Prepares the datasets for train, validation and testing

val_dataset=build_dataset(x_val,y_val,repeat=False,batch=64)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=64)

BATCH_SIZE=32
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_train,y_train,batch=BATCH_SIZE)

# input shape for the model
input_shape=train_dataset.element_spec[0].shape[1:]

print(input_shape)

(224, 224, 3)


In [183]:
# load the best model, trained before
model_full_trained = keras.models.load_model("model/model_baseline.keras")

model_al = keras.models.load_model("model/model_al.keras")

# Test full model

In [184]:
y_pred_full = []
y_true_full = []

class_names = {0: 'No DR', 1: 'DR'}

# Iterate over the test dataset
for x_batch, y_batch in test_dataset:
    # Predict probabilities for each batch
    y_test_proba = model_full_trained.predict(x_batch)

    # Convert probabilities to class labels (for softmax output)
    y_pred_full.extend(np.argmax(y_test_proba, axis=1))

    # Convert true labels from one-hot encoding to class labels
    y_true_full.extend(np.argmax(y_batch.numpy(), axis=1))

# Convert lists to numpy arrays
y_pred_full = np.array(y_pred_full)
y_true_full = np.array(y_true_full)

# Transform 0 and 1 into class names
y_pred_full_names = [class_names[label] for label in y_pred_full]
y_true_full_names = [class_names[label] for label in y_true_full]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


# Test Active Learning Model

In [185]:
y_pred_al = []
y_true_al = []

# Iterate over the test dataset
for x_batch, y_batch in test_dataset:
    # Predict probabilities for each batch
    y_test_proba_al = model_al.predict(x_batch)

    # Convert probabilities to class labels (for softmax output)
    y_pred_al.extend(np.argmax(y_test_proba_al, axis=1))

    # Convert true labels from one-hot encoding to class labels
    y_true_al.extend(np.argmax(y_batch.numpy(), axis=1))

# Convert lists to numpy arrays
y_pred_al = np.array(y_pred_al)
y_true_al = np.array(y_true_al)

# Transform 0 and 1 into class names
y_pred_al_names = [class_names[label] for label in y_pred_al]
y_true_al_names = [class_names[label] for label in y_true_al]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


# Metrics of full trained model

In [186]:
print(model_full_trained.evaluate(test_dataset, verbose=0,return_dict=True))
conf_matrix = confusion_matrix(y_true_full, y_pred_full)
print("\nConfusion Matrix:\n", conf_matrix)

# Print classification report
print("\nClassification Report:\n", classification_report(y_true_full_names, y_pred_full_names, target_names=list(class_names.values())))

{'categorical_accuracy': 0.9535518884658813, 'loss': 0.13201043009757996}

Confusion Matrix:
 [[188   7]
 [ 10 161]]

Classification Report:
               precision    recall  f1-score   support

       No DR       0.96      0.94      0.95       171
          DR       0.95      0.96      0.96       195

    accuracy                           0.95       366
   macro avg       0.95      0.95      0.95       366
weighted avg       0.95      0.95      0.95       366



# Metrics of AL model

In [187]:
print(model_al.evaluate(test_dataset, verbose=0,return_dict=True))

conf_matrix = confusion_matrix(y_true_al, y_pred_al)
print("\nConfusion Matrix:\n", conf_matrix)

# Print classification report
print("\nClassification Report:\n", classification_report(y_true_al_names, y_pred_al_names, target_names=list(class_names.values())))

{'categorical_accuracy': 0.9535518884658813, 'loss': 0.2187982201576233}

Confusion Matrix:
 [[188   7]
 [ 10 161]]

Classification Report:
               precision    recall  f1-score   support

       No DR       0.96      0.94      0.95       171
          DR       0.95      0.96      0.96       195

    accuracy                           0.95       366
   macro avg       0.95      0.95      0.95       366
weighted avg       0.95      0.95      0.95       366

